In [1]:
!pip install kaggle

In [2]:
import os
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from PIL import Image

In [5]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [6]:
#fetching dataset using api
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia

Dataset URL: https://www.kaggle.com/datasets/paultimothymooney/chest-xray-pneumonia
License(s): other
chest-xray-pneumonia.zip: Skipping, found more recently modified local copy (use --force to force download)


In [7]:
from zipfile import ZipFile
with ZipFile("/content/chest-xray-pneumonia.zip",'r') as zip_ref:
  zip_ref.extractall()
  print("Dataset Extracted")

Dataset Extracted


In [8]:
dataset_path = "/content/chest_xray"


In [9]:

train_dir = dataset_path + "/train"
val_dir = dataset_path + "/val"
test_dir = dataset_path + "/test"

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.3,
    horizontal_flip=True,
    brightness_range=[0.8, 1.2],
    validation_split=0.2
)

# Load Training Data (80%)
train_data = train_datagen.flow_from_directory(
    train_dir, target_size=(224, 224), batch_size=32, class_mode='binary', subset='training')

# Load Validation Data (20%)
val_data = train_datagen.flow_from_directory(
    train_dir, target_size=(224, 224), batch_size=32, class_mode='binary', subset='validation')

# Load Test Data (Only Normalization)
test_datagen = ImageDataGenerator(rescale=1./255)
test_data = test_datagen.flow_from_directory(
    test_dir, target_size=(224, 224), batch_size=32, class_mode='binary')


Found 4173 images belonging to 2 classes.
Found 1043 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [11]:
# Load Pretrained MobileNetV2
base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
base_model.trainable = False  # Freeze base model

# Custom Classification Layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(1, activation='sigmoid')(x)  # Binary Classification

# Create Model
model = Model(inputs=base_model.input, outputs=output)

# Compile Model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Show Model Summary
model.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1             │ (None, 224, 224, 3)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Conv1 (Conv2D)            │ (None, 112, 112, 32)   │            864 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bn_Conv1                  │ (None, 112, 112, 32)   │            128 │ Conv1[0][0]            │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ Conv1_relu (ReLU)         │ (None, 112, 112, 32)   │              0 │ bn_Conv1[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise   │ (None, 112, 112, 32)   │            288 │ Conv1_relu[0][0]       │
│ (DepthwiseConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise_… │ (None, 112, 112, 32)   │            128 │ expanded_conv_depthwi… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_depthwise_… │ (None, 112, 112, 32)   │              0 │ expanded_conv_depthwi… │
│ (ReLU)                    │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_project     │ (None, 112, 112, 16)   │            512 │ expanded_conv_depthwi… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ expanded_conv_project_BN  │ (None, 112, 112, 16)   │             64 │ expanded_conv_project… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_expand (Conv2D)   │ (None, 112, 112, 96)   │          1,536 │ expanded_conv_project… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_expand_BN         │ (None, 112, 112, 96)   │            384 │ block_1_expand[0][0]   │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_expand_relu       │ (None, 112, 112, 96)   │              0 │ block_1_expand_BN[0][… │
│ (ReLU)                    │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_pad               │ (None, 113, 113, 96)   │              0 │ block_1_expand_relu[0… │
│ (ZeroPadding2D)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block_1_depthwise         │ (None, 56, 56, 96)     │            864 │ block_1_pad[0][0]      │
│ (DepthwiseConv2D)         │                        │                │                        │
├──────────────────────

 Total params: 2,422,081 (9.24 MB)

 Trainable params: 164,097 (641.00 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [ ]:
# Train the Model
class_weights = {0: 1, 1: 2}  # Give Pneumonia class more weight
history=model.fit(train_data, validation_data=val_data, epochs=10, class_weight=class_weights)

#history = model.fit(train_data, validation_data=val_data, epochs=10)


Epoch 1/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 342s 3s/step - accuracy: 0.9114 - loss: 0.2684 - val_accuracy: 0.9329 - val_loss: 0.1569
Epoch 2/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 307s 2s/step - accuracy: 0.9188 - loss: 0.2389 - val_accuracy: 0.9348 - val_loss: 0.1618
Epoch 3/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 306s 2s/step - accuracy: 0.9404 - loss: 0.2097 - val_accuracy: 0.9386 - val_loss: 0.1427
Epoch 4/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 296s 2s/step - accuracy: 0.9350 - loss: 0.2131 - val_accuracy: 0.9348 - val_loss: 0.1631
Epoch 5/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 307s 2s/step - accuracy: 0.9380 - loss: 0.2088 - val_accuracy: 0.9329 - val_loss: 0.1631
Epoch 6/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 338s 3s/step - accuracy: 0.9300 - loss: 0.2249 - val_accuracy: 0.9300 - val_loss: 0.1563
Epoch 7/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 321s 2s/step - accuracy: 0.9401 - loss: 0.1908 - val_accuracy: 0.9396 - val_loss: 0.1434
Epoch 8/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 295s 2s/step - accuracy: 0.9457 - loss: 0.1767 - val_accu

In [ ]:
# Evaluate the Model
loss, accuracy = model.evaluate(test_data)
print(f"Test Accuracy: {accuracy * 100:.2f}%")


20/20 ━━━━━━━━━━━━━━━━━━━━ 30s 1s/step - accuracy: 0.8807 - loss: 0.3211
Test Accuracy: 88.94%


In [ ]:
# Load the trained model
model = tf.keras.models.load_model("pneumonia_detector.keras")


/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 6 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [ ]:
# Load the trained pneumonia detection model
model = tf.keras.models.load_model("pneumonia_detector.keras")  # Ensure the file is in the same folder

# Define class labels
class_names = ["Normal", "Pneumonia"]


In [ ]:


# Load the trained model (Ensure the file exists in the same folder)
model = tf.keras.models.load_model("pneumonia_detector.keras")

# Define class labels
class_names = ["Normal", "Pneumonia"]

def predict_pneumonia(img_path):
    try:
        # Check if the file exists
        if not os.path.exists(img_path):
            print(f"Error: File not found at {img_path}")
            return

        # Load and preprocess the image
        img = Image.open(img_path).convert("RGB")  # Convert grayscale to RGB
        img = img.resize((224, 224))  # Resize to fit the model input size
        img_array = np.array(img) / 255.0  # Normalize pixel values
        img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

        # Make prediction
        prediction = model.predict(img_array)[0][0]  # Get model output
        predicted_class = class_names[int(prediction > 0.6)]
        confidence = prediction * 100 if prediction > 0.6 else (1 - prediction) * 100

        # Print the result
        print(f"Prediction: {predicted_class} (Confidence: {confidence:.2f}%)")

    except Exception as e:
        print(f"Error processing image: {e}")

# Example Usage:
image_path = input("Enter the X-ray image path: ").strip()
predict_pneumonia(image_path)


Enter the X-ray image path: /content/chest_xray/test/PNEUMONIA/person100_bacteria_475.jpeg
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Prediction: Pneumonia (Confidence: 89.17%)
